In [3]:
# MySQL 에서 df 가져오는 코드 
import pymysql
import pandas as pd
import matplotlib as mpl
import matplotlib.font_manager as fm
pd.set_option('display.max_columns', None)

# 폰트 설정 
# font_name = fm.FontProperties(fname="/Library/Fonts/AppleGothic.ttf").get_name()
font_name = fm.FontProperties(fname='C:/Windows/Fonts/malgun.ttf' ).get_name()
mpl.rc('font', family=font_name)

# 데이터베이스에 연결 
cnx = pymysql.connect(user=')

query = 'SELECT * FROM zigbang_project.zb_table;'

df = pd.read_sql(query, cnx)

# Close the database connection
cnx.close()

c:\Users\USER_20211027\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [9]:
# 전월세 이자율 7% 기준을 통합
df['deposit+rent']= df['deposit']*0.07/12 + df['rent']

In [15]:
df['deposit+rent'][df['deposit']==5000]

128      84.166667
200      64.166667
220      39.166667
236      89.166667
245      54.166667
           ...    
34582    84.166667
34595    99.166667
34605    74.166667
34644    29.166667
34651    29.166667
Name: deposit+rent, Length: 2587, dtype: float64

In [29]:
# df.loc[34651,['deposit','rent','deposit+rent']]
df.loc[128,:]

id                                  36598666
service_type                              원룸
address1                      서울시 영등포구 양평동1가
address2                                 136
_floor                                     7
                                   ...      
ppltn_tot_transfer_in_cnt              79309
ppltn_tot_transfer_out_cnt             68526
ppltn_Net_migration_cnt                10783
safety_idx                                16
deposit+rent                       84.166667
Name: 128, Length: 154, dtype: object

In [35]:
a =df.columns[0:24].tolist

In [43]:
# 관리비 포함 옵션 갯수
df['manage_cost_inc']
df['manage_cost_inc_num'] = df['manage_cost_inc'].str.split(',').apply(len)

In [46]:
df['near_subways']
df['near_subways_num'] = df['near_subways'].str.split(',').apply(len)

In [47]:
# df['near_subways_num']

0        3
1        2
2        2
3        1
4        2
        ..
34677    1
34678    1
34679    2
34680    1
34681    1
Name: near_subways_num, Length: 34682, dtype: int64

In [49]:
df['options_num'] = df['options'].str.split(',').apply(len)

In [58]:
# 방향 점수화 해서 넣는거 어떰?
df['room_direction_text'].values

array(['동향', '남향', '서향', ..., '동향', '남서향', '남향'], dtype=object)

In [76]:
df_selected = df[['_floor', 'size', 'manage_cost', 'manage_cost_inc_num', 'near_subways_num', 'options_num','deposit+rent', 'parking', 'elevator']]

In [72]:
df['parking']

0        1
1        0
2        1
3        1
4        0
        ..
34677    1
34678    1
34679    1
34680    1
34681    1
Name: parking, Length: 34682, dtype: int64

In [78]:
# y = 'rent', x 변수 81개 
# 회귀분석 
import statsmodels.api as sm

X = df_selected.drop('deposit+rent', axis=1)
y = df_selected['deposit+rent']

X1 = sm.add_constant(X)

# 회귀 모델 생성 
model = sm.OLS(y, X1)

results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:           deposit+rent   R-squared:                       0.466
Model:                            OLS   Adj. R-squared:                  0.465
Method:                 Least Squares   F-statistic:                     3775.
Date:                Sun, 14 May 2023   Prob (F-statistic):               0.00
Time:                        21:27:03   Log-Likelihood:            -1.7057e+05
No. Observations:               34682   AIC:                         3.411e+05
Df Residuals:                   34673   BIC:                         3.412e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                  -1.7504    

In [79]:
df_selected.shape

(34682, 9)